In [9]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import nltk
from nltk.corpus import stopwords 

In [12]:
# data_path = '/Users/zhanghan/mygithub/nlp-beginner/task1-sentiment-analysis-on-movie-reviews/train.tsv'
# df = load_data(data_path)
# sentences = df['Phrase']

## test

In [63]:
data_t = np.array(pd.read_csv('/Users/zhanghan/mygithub/nlp-beginner/task1-sentiment-analysis-on-movie-reviews/train.tsv',
                            sep='\t', header=0, nrows=5))
# X_text = df_t[?]#取出phrase列

print(df_t)
print('---')
print(sentences_t)
# print(len(sentences_t))
print('---')
print(data_t[0][2])


[[1 1
  'A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .'
  1]
 [2 1
  'A series of escapades demonstrating the adage that what is good for the goose'
  2]
 [3 1 'A series' 2]
 [4 1 'A' 2]
 [5 1 'series' 2]]
---
[[1 1
  'A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .'
  1]
 [2 1
  'A series of escapades demonstrating the adage that what is good for the goose'
  2]]
---
A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .


## load data (fin)

In [4]:
def load_data(data_path):
    '''load data from tsv file'''
    df = pd.read_csv(data_path, sep='\t', header=0, index_col='id')
    return np.array(df)

## visualize data

In [5]:
def visualize_data(data):#还可以改改 或者不写也行
    '''visualize original data'''
    
    return 1

## clean text (fin)

In [47]:
def remove_stopword(data_orig):
    '''
    remove stopwords in the wordlist
    
    data: array
    return an array
    '''
    data_filtered = data_orig.copy()   
    for i in range(data_orig.shape[0]):# 通过遍历修改data[i][2]的字符串
        filtered_words = [word.lower() for word in data_orig[i][2].split() if word not in stopwords.words('english')]
        l = ' '.join(filtered_words)
        data_filtered[i][2] = ' '.join(filtered_words) 
    return data_filtered


# test
# d = np.array([[1,1,'A series of escapades demonstrating the a',1],
#                [2,1, 'A series of escapades demonstrating the adage that what is good for the goose', 2],
#                [3, 1, 'A series' ,2],
#                [4, 1, 'A', 2],
#                [5, 1, 'series', 2]])
# d_f = remove_stopword(d)
# d_f

array([['1', '1', 'a series escapades demonstrating', '1'],
       ['2', '1', 'a series escapades demonstrating adage good goose',
        '2'],
       ['3', '1', 'a series', '2'],
       ['4', '1', 'a', '2'],
       ['5', '1', 'series', '2']], dtype='<U77')

## make dictionary=tokenize

In [57]:
# 之后要把这里的data改成X，不要整个data进入
def make_dict_bow(data_filtered, text_col=2):
    '''
    make a bow dictionary based on all the words in the text
    return a set that contains a word list mapped with numbers
    
    data: filtered data with all columns
    text_col: text column index
    '''
    # make a non-repetitive word set
    word_set = set()
    for row in data_filtered:
        for word in row[text_col].split():
            word_set.add(word) # ('a','b')
            
    # map each string with an index number
    word_size = len(word_set)
    dict_bow = dict(zip(word_set,range(word_size))) # {'a':0,'b':1}
                    
    return dict_bow

                    
def make_dict_ng(data_filtered, text_col=2, ngram = 3):
    '''
    make a n-gram dict
    
    data_filtered: sentences array
    ngram: n-gram number
    '''
    # "在一般的文本分类任务中,N 取 3 就足够了
    
    # make a non-repetitive word set 
    word_set = set()
    for row in data_filtered:
        wordlist_row = row[text_col].split()
        for n in range(ngram):
            for i in range(len(wordlist_row) - n):
                word_set.add(' '.join(wordlist_row[ i : i + ngram ])) # ('a','a b')
        
    # map each string with an index number
    word_size = len(word_set)
    dict_ng = dict(zip(word_set,range(word_size))) # {'a':0,'b':1}
    
    return dict_ng

# test passed!
# d2 = np.array([['1', '1', 'a series escapades demonstrating', '1'],
#        ['2', '1', 'a series escapades demonstrating adage good goose',
#         '2'],
#        ['3', '1', 'a series', '2'],
#        ['4', '1', 'a', '2'],
#        ['5', '1', 'series', '2']], dtype='<U77')
# print('d2_bow: ', make_dict_bow(d2,2))
# print('d2_ng: ', make_dict_ng(d2))

d2_bow:  {'goose': 0, 'a': 1, 'good': 2, 'adage': 3, 'escapades': 4, 'series': 5, 'demonstrating': 6}
d2_ng:  {'escapades demonstrating adage': 0, 'goose': 1, 'a': 2, 'good goose': 3, 'a series escapades': 4, 'series escapades demonstrating': 5, 'demonstrating adage good': 6, 'adage good goose': 7, 'series': 8, 'a series': 9, 'demonstrating': 10, 'escapades demonstrating': 11}


## data -> vec

In [81]:
def vectorize_bow(data_filtered, dict_bow, text_col=2): 
    '''
    data_filtered: array, filtered data
    return an array of vectorized features
    '''
    features_bow = np.zeros((data_filtered.shape[0],len(dict_bow)))
    for i in range(data_filtered.shape[0]): # array中的每个句子：
        phrase_i = data_filtered[i][text_col]
        wordlist = phrase_i.split()
        for word in wordlist: # for每个句子中的每个单词：
            if word in dict_bow:# if这个单词在字典中：
                j = dict_bow[word]
                features_bow[i][j] += 1 # features对应dict相同index的位置变为1
    return features_bow


def vectorize_ng(data_filtered, dict_ng, text_col=2, ngram=3): # N-gram：ng_vectorizer.transform(corpus)
    '''
    data_filtered: array, filtered data
    return an array of vectorized features
    '''
    features_ng = np.zeros((data_filtered.shape[0],len(dict_ng)))
    for i in range(data_filtered.shape[0]): # for array中的每个句子
        phrase_i = data_filtered[i][text_col]
        wordlist = phrase_i.split()
        for n in range(ngram):
            for index in range(len(wordlist)-n): # for每个句子中的每个word_ng：
                word_ng = ' '.join(wordlist[ index : index + n+1 ])
                # print(index, word_ng)
                if word_ng in dict_ng: # if这个word_ng在字典中：
                    j = dict_ng[word_ng]
                    features_ng[i][j] += 1 # features对应dict相同index的位置+1
    return features_ng

# test passed!
# a3 = np.array([['1', '1', 'a series escapades demonstrating', '1'],
#        ['2', '1', 'a series escapades demonstrating adage good goose',
#         '2'],
#        ['3', '1', 'a series', '2'],
#        ['4', '1', 'a', '2'],
#        ['5', '1', 'series', '2']], dtype='<U77')
# print('original data: \n',a3,'\n')

# dict_bow_t = make_dict_bow(a3,2)
# dict_ng_t = make_dict_ng(a3)

# f_t_bow = vectorize_bow(a3, dict_bow_t)
# f_t_ng = vectorize_ng(a3, dict_ng_t)
# print(dict_bow_t, '\n', '*'*25, '\n',f_t_bow)
# print('\n')
# print(dict_ng_t, '\n', '*'*25, '\n', f_t_ng)

original data: 
 [['1' '1' 'a series escapades demonstrating' '1']
 ['2' '1' 'a series escapades demonstrating adage good goose' '2']
 ['3' '1' 'a series' '2']
 ['4' '1' 'a' '2']
 ['5' '1' 'series' '2']] 

{'goose': 0, 'a': 1, 'good': 2, 'adage': 3, 'escapades': 4, 'series': 5, 'demonstrating': 6} 
 ************************* 
 [[0. 1. 0. 0. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [0. 1. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]]


{'escapades demonstrating adage': 0, 'goose': 1, 'a': 2, 'good goose': 3, 'a series escapades': 4, 'series escapades demonstrating': 5, 'demonstrating adage good': 6, 'adage good goose': 7, 'series': 8, 'a series': 9, 'demonstrating': 10, 'escapades demonstrating': 11} 
 ************************* 
 [[0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


## train (get model+gradient descent)

cost function:

$$
\begin{aligned}
\mathcal{R}(w)
	&=-\frac{1}{N}\displaystyle \sum^{N}_{n=1}\mathcal{L}(y^{(n)},\hat y^{(n)})\\
	&=-\frac{1}{N}\displaystyle \sum^{N}_{n=1}{\left(y^{(n)}\right)^T\log \hat y^{(n)}\\}
\end{aligned}\tag{3.6}
$$

gradient:

$$
\frac{\partial \mathcal{R}(w)}{\partial w}=-\frac{1}{N}\sum_{n=1}^{N}{x^{(n)}(y^{(n)}-\hat y^{(n)})}\tag{3.7}
$$


In [69]:
def softmax(z):
    '''compute the softmax output of z = wx + b'''
    ez = np.exp(z)
    sm = ez / np.sum(ez)
    return sm
    

def compute_cost(X, y, W, b):
    '''
    compute the total cost of given parameters
    
    Args:
        X (array, (m,n)): m training examples, n features
        y (array, (n,))
    '''
    
    m,n = X.shape
    y_hat = np.dot(W,x) + b
    cost = np.dot(y, np.log(y_hat)) * (-1) / m
    
    return cost

# 遗留问题：bias的处理？itertimes和epoch？mini-batch & sgd的实现方法？
def train(X_train, y_train, W_init, b_init, alpha, itertimes, gd='batch'):
    '''
    learn the parameters
    
    Args:
        X_train (array, (m,n)): m training examples, n features
        y_train (array, (m,))
        W_init (array, (n,))
        alpha (float): learning rate
        itertimes (int): iteration times
        gd (string): method for gradient descent. 'batch', 'stochastic' or 'mini-batch'
    Return
        W (array, (n,)): parameters
        cost_hist (list): a list of cost for plotting
    '''
    
    m,n = X.shape
    
    # compute gradient
    dj_dw = np.zeros((n,))                            #(n,)
    dj_db = 0.                                        #scalar
    for i in range(m):
        y_hat = softmax(np.dot(X_train[i],W)+b_init)         #(n,)(n,)=scalar
        err_i  = y_hat_i  - y[i]                             #scalar
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * X_train[i,j]      #scalar
        dj_db += err_i
    dj_dw = dj_dw/m                                         #(n,)       
    dj_db = dj_db/m  
    
    # gradient descent: SGD/batch/mini-batch
    # 还差minibatch和sgd 要看一下书
    cost_hist = []
    W = copy.deepcopy(W_in)
    for i in range(itertimes):
        # Update Parameters using w, b, alpha and gradient
        W = W - alpha * dj_dw
        b_init = W - alpha * dj_db
      
        # Save cost at each iteration
        if i<100000:      # prevent resource exhaustion 
            cost_hist.append(compute_cost(X_train, y_train, W, b_init) )
    
    return W, cost_hist




SyntaxError: invalid syntax (2523655359.py, line 1)

### #prediction

In [ ]:
def save_data():



def accuracy():

def plot_result():